In [9]:
import pandas as pd
import numpy as np
import networkx as nx

In [23]:
df = pd.read_excel('plantilla_metro.xlsx', index_col=None)

In [24]:
print(df.head()) 
print(df.columns)

  Estación Origen     Estación Destino Color  Logitud de Estación
0       San Pablo              Neptuno  Rojo                  1.0
1         Neptuno            Pajaritos  Rojo                  1.0
2       Pajaritos            Las Rejas  Rojo                  1.0
3       Las Rejas              Ecuador  Rojo                  1.0
4         Ecuador  San Alberto Hurtado  Rojo                  1.0
Index(['Estación Origen', 'Estación Destino', 'Color', 'Logitud de Estación'], dtype='object')


In [27]:
METRO = nx.from_pandas_edgelist(df,source='Estación Origen',target='Estación Destino',edge_attr='Logitud de Estación')
print()

In [28]:
METRO.nodes()

NodeView(('San Pablo', 'Neptuno', 'Pajaritos', 'Las Rejas', 'Ecuador', 'San Alberto Hurtado', 'Universidad de Santiago', 'Estación Central', 'ULA', 'República', 'Los Héroes', 'La Moneda', 'Universidad de Chile', 'Santa Lucía', 'Universidad Católica', 'Baquedano', 'Salvador', 'Manuel Montt', 'Pedro de Valdivia', 'Los Leones', 'Tobalaba', 'El Golf', 'Alcántara', 'Escuela Militar', 'Manquehue', 'Hernando de Magallanes', 'Los Dominicos', 'Vespucio Norte', 'Zapadores', 'Dorsal', 'Einstein', 'Cementerios', 'Patronato', 'Puente Cal y Canto', 'Santa Ana', 'Toesca', "Parque O'Higgins", 'Rondizzoni', 'Franklin', 'El Llano', 'San Miguel', 'Lo Vial', 'Departamental', 'Ciudad del Niño', 'Lo Ovalle', 'El Parrón', 'La Cisterna', 'Los Libertadores', 'Cardenal Caro', 'Vivaceta', 'Conchalí', 'Plaza Chacabuco', 'Hospitales', 'Plaza de Armas', 'Santa Isabel', 'Irarrázaval', 'Monseñor Eyzaguirre', 'Ñuñoa', 'Chile España', 'Villa Frei', 'Plaza Egaña', 'Los Orientales', 'Fernando Castillo Velasco', 'Cristóba

In [29]:
METRO.edges()

EdgeView([('San Pablo', 'Neptuno'), ('San Pablo', 'Pudahuel'), ('San Pablo', 'Lo Prado'), ('Neptuno', 'Pajaritos'), ('Pajaritos', 'Las Rejas'), ('Las Rejas', 'Ecuador'), ('Ecuador', 'San Alberto Hurtado'), ('San Alberto Hurtado', 'Universidad de Santiago'), ('Universidad de Santiago', 'Estación Central'), ('Estación Central', 'ULA'), ('ULA', 'República'), ('República', 'Los Héroes'), ('Los Héroes', 'La Moneda'), ('Los Héroes', 'Santa Ana'), ('Los Héroes', 'Toesca'), ('La Moneda', 'Universidad de Chile'), ('Universidad de Chile', 'Santa Lucía'), ('Universidad de Chile', 'Puente Cal y Canto'), ('Universidad de Chile', 'Plaza de Armas'), ('Santa Lucía', 'Universidad Católica'), ('Universidad Católica', 'Baquedano'), ('Baquedano', 'Salvador'), ('Baquedano', 'Bellas Artes'), ('Baquedano', 'Parque Bustamante'), ('Salvador', 'Manuel Montt'), ('Manuel Montt', 'Pedro de Valdivia'), ('Pedro de Valdivia', 'Los Leones'), ('Los Leones', 'Tobalaba'), ('Los Leones', 'Inés de Suárez'), ('Tobalaba', 'E

In [32]:
METRO.order()

115

In [31]:
for x in METRO.nodes():
    if METRO.degree(x) > 2:
        print(x)

San Pablo
Los Héroes
Universidad de Chile
Baquedano
Los Leones
Tobalaba
Puente Cal y Canto
Santa Ana
Franklin
Plaza de Armas
Santa Isabel
Irarrázaval
Ñuñoa
Plaza Egaña
Vicuña Mackenna
Vicente Valdés
Ñuble


In [36]:
djk_path=nx.dijkstra_path(METRO, source='San Pablo', target='San Ramón' )
djk_path

['San Pablo',
 'Lo Prado',
 'Blanqueado',
 'Gruta de Lourdes',
 'Quinta Normal',
 'Cumming',
 'Santa Ana',
 'Los Héroes',
 'Toesca',
 "Parque O'Higgins",
 'Rondizzoni',
 'Franklin',
 'El Llano',
 'San Miguel',
 'Lo Vial',
 'Departamental',
 'Ciudad del Niño',
 'Lo Ovalle',
 'El Parrón',
 'La Cisterna',
 'Santa Rosa',
 'San Ramón']

In [ ]:
ruta1 = METRO.subgraph(['San Pablo',
 'Lo Prado',
 'Blanqueado',
 'Gruta de Lourdes',
 'Quinta Normal',
 'Cumming',
 'Santa Ana',
 'Los Héroes',
 'Toesca',
 "Parque O'Higgins",
 'Rondizzoni',
 'Franklin',
 'El Llano',
 'San Miguel',
 'Lo Vial',
 'Departamental',
 'Ciudad del Niño',
 'Lo Ovalle',
 'El Parrón',
 'La Cisterna',
 'Santa Rosa',
 'San Ramón'])
nx.draw(ruta1, with_labels=True)